In [ ]:
# Install requirements, uncomment to run
# !pip install-U geopandas pillow label-studio-converter

# Formatting SavMap dataset
Formatting the savmap dataset so it can be imported in label studio

In [1]:
import geopandas as gdp
from PIL import Image
from tqdm import tqdm
import os
from pathlib import Path

In [4]:
polygons = gdp.read_file("../data/savmap_dataset_v2/savmap_annotations_2014.geojson")

In [ ]:
# create directories
Path("../data/savmap_dataset_v2/images").mkdir(exist_ok=True,parents=True)
Path("../data/savmap_dataset_v2/labels").mkdir(exist_ok=True,parents=True)

! mv ../data/savmap_dataset_v2/*.JPG ../data/savmap_dataset_v2/images

In [8]:
# get bounding boxs
for cat in ['x_min', 'y_min', 'x_max', 'y_max','width','height','x','y']:
    polygons[cat] = None
for i in range(len(polygons)):
    x_min, y_min, x_max, y_max = polygons['geometry'].iloc[i].bounds
    image_path = f"../data/savmap_dataset_v2/images/{polygons['IMAGEUUID'].iloc[i]}.JPG"
    width, height = Image.open(image_path).size
    polygons['x_min'].iat[i] = x_min
    polygons['x_max'].iat[i] = x_max
    polygons['y_min'].iat[i] = y_min
    polygons['y_max'].iat[i] = y_max
    polygons['x'].iat[i] = 0.5*(x_max+x_min)
    polygons['y'].iat[i] = 0.5*(y_max+y_min)
    polygons['width'] = width
    polygons['height'] = height

polygons['bbox_w'] = polygons['x_max'] - polygons['x_min']
polygons['bbox_h'] = polygons['y_max'] - polygons['y_min']
polygons['class'] = 0

In [9]:
polygons.head(1)

,IMAGEUUID,TAGUUID,geometry,x_min,y_min,x_max,y_max,width,height,x,y,bbox_w,bbox_h,class
0,f77f4af5a1344b9086b307d2b4ba61ff,a9b3a2325dbe4a208bc3ae37eeb8e1e1,"POLYGON ((1197 568, 1186 568, 1179 582, 1190 5...",1179.0,568.0,1230.0,597.0,4608,3456,1204.5,582.5,51.0,29.0,0


In [10]:
# convert to yolo format
def save_df_as_yolo(df_annotation:gdp.GeoDataFrame,dest_path_labels:str='../data/savmap_dataset_v2/labels'):
    
    cols = ['class','x','y','bbox_w','bbox_h']
    for col in cols:
        assert df_annotation[col].isna().sum()<1,'there are NaN values. Check out.'
        # df_annotation[col] = df_annotation[col].apply(int)

    # normalize values
    df_annotation.loc[:,'x'] = df_annotation['x']/df_annotation['width']
    df_annotation.loc[:,'y'] = df_annotation['y']/df_annotation['height']
    df_annotation.loc[:,'bbox_w'] = df_annotation['bbox_w']/df_annotation['width']
    df_annotation.loc[:,'bbox_h'] = df_annotation['bbox_h']/df_annotation['height']
    
    for image_name,df in tqdm(df_annotation.groupby('IMAGEUUID'),desc='Saving yolo labels'):
        txt_file = f'{image_name}.txt'
        df[cols].to_csv(os.path.join(dest_path_labels,txt_file),sep=' ',index=False,header=False)

In [11]:
save_df_as_yolo(df_annotation=polygons)

Saving yolo labels:  29%|██▉       | 192/654 [00:00<00:00, 629.79it/s]

Saving yolo labels: 100%|██████████| 654/654 [00:01<00:00, 635.31it/s]


In [17]:
!label-studio-converter import yolo -i ../data/savmap_dataset_v2 -o ../data/savmap_dataset_v2/labelstudio.json --image-root-url "/data/local-files/?d=savmap_dataset_v2/images" --image-ext ".JPG" --out-type "annotations"

INFO:root:Reading YOLO notes and categories from /workspaces/datalabeling/data/savmap_dataset_v2
INFO:root:Found 1 categories
INFO:root:Converting labels from /workspaces/datalabeling/data/savmap_dataset_v2/labels
INFO:root:image extensions->, ['.JPG']
INFO:root:Saving Label Studio JSON to /workspaces/datalabeling/data/savmap_dataset_v2/labelstudio.json

  1. Create a new project in Label Studio
  2. Use Labeling Config from "/workspaces/datalabeling/data/savmap_dataset_v2/labelstudio.label_config.xml"
  3. Setup serving for images [e.g. you can use Local Storage (or others):
     https://labelstud.io/guide/storage.html#Local-storage]
  4. Import "/workspaces/datalabeling/data/savmap_dataset_v2/labelstudio.json" to the project

